## Model Ensemble

Having fitted several models, we now combine them to try squeeze out any further predictive performance. This is achieved through the use of SKLearn's  `StackingClassifier`.

In [ ]:
models = {'xgb':xgb_cl,
          'bag': bag_clf,
          'lgb': lgb_clf,
          'rf':rf_final,
          'AdaBoost': adaModel}
stacked_model = stack_models(*models.items(), cv_folds=3)
stacked_model.fit(X_train, y_train)
models['stacked'] = stacked_model

In [ ]:
# Create a dataframe to show the performance of each model
performance = pd.DataFrame(y_test)
performance['stacking_prediction'] = stacked_model.predict(X_test)
for m in stacked_model.named_estimators_:
        performance[m] = stacked_model.named_estimators_[m].predict(X_test)

In [ ]:
# See how each of our models correlate with our target
print("Correlations with target column")
print(temp.corr()['isFraud'])

In [ ]:
from matplotlib import pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

# evaluate models using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=1)
	scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
	return scores

In [ ]:
# evaluate the models and store results
results, names = list(), list()
for name, m in models.items():
	scores = evaluate_model(m, X_test, y_test)
	results.append(scores)
	names.append(name)
	print(f'>{name} {mean(scores):0.3f} ({std(scores):0.3f})')

# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
from utils.diagnostics import get_diagnostics
get_diagnostics(*models.items(), y_true=y_test, X=X_test)